In [0]:
import pandas as pd
from datetime import datetime
from pyspark.sql.functions import avg, countDistinct, to_date, datediff, col, max, row_number
from pyspark.sql.window import Window

In [0]:
base_path = "s3://columbia-gr5069-main/raw/"
df_pit = spark.read.option("header", True).csv(base_path + "pit_stops.csv")
df_results = spark.read.option("header", True).csv(base_path + "results.csv")
df_drivers = spark.read.option("header", True).csv(base_path + "drivers.csv")
df_races = spark.read.option("header", True).csv(base_path + "races.csv")
df_lap_times = spark.read.option("header", True).csv(base_path + "lap_times.csv")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-2850152597172470>, line 2
      1 base_path = "s3://columbia-gr5069-main/raw/"
----> 2 df_pit = spark.read.option("header", True).csv(base_path + "pit_stops.csv")
      3 df_results = spark.read.option("header", True).csv(base_path + "results.csv")
      4 df_drivers = spark.read.option("header", True).csv(base_path + "drivers.csv")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:835, in DataFrameReader.csv(self, path, schema, sep, encoding, quote,

In [0]:
base_path = "/Workspace/Users/hh3110@columbia.edu/f1/"

df_pit = pd.read_csv(base_path + "pit_stops.csv")
df_results = pd.read_csv(base_path + "results.csv")
df_drivers = pd.read_csv(base_path + "drivers.csv")
df_races = pd.read_csv(base_path + "races.csv")
df_lap_times = pd.read_csv(base_path + "lap_times.csv")

In [0]:
# Q1: Average pit stop time per driver per race
avg_pit = df_pit.groupby(["raceId", "driverId"])["milliseconds"].mean().reset_index(name="avg_pit_time")
print("Q1: Average Pit Stop Time")
print(avg_pit.head)


Q1: Average Pit Stop Time
<bound method NDFrame.head of       raceId  driverId  avg_pit_time
0        841         1       23213.0
1        841         2       24046.0
2        841         3       23716.0
3        841         4       24055.0
4        841         5       24865.0
...      ...       ...           ...
5570    1144       855       25894.5
5571    1144       857       28178.0
5572    1144       859       26099.0
5573    1144       861       24941.0
5574    1144       862       22080.0

[5575 rows x 3 columns]>


In [0]:
# Q2: Rank by race winner
pit_result = avg_pit.merge(df_results[["raceId", "driverId", "positionOrder"]], on=["raceId", "driverId"])
pit_result_sorted = pit_result.sort_values(by=["raceId", "positionOrder"])
print("Q2: Average Pit Time Ranked by Position")
print(pit_result_sorted)

Q2: Average Pit Time Ranked by Position
      raceId  driverId  avg_pit_time  positionOrder
11       841        20  23319.500000              1
0        841         1  23213.000000              2
17       841       808  25109.000000              3
3        841         4  24055.000000              4
9        841        17  24058.666667              5
...      ...       ...           ...            ...
5574    1144       862  22080.000000             15
5560    1144       825  23236.500000             16
5572    1144       859  26099.000000             17
5559    1144       822  41144.000000             18
5573    1144       861  24941.000000             19

[5575 rows x 4 columns]


In [0]:
# Q3: Fill missing driver codes (if any)
print("Missing codes (before):")
print(df_drivers[df_drivers["code"].isnull()][["driverId", "forename", "surname"]])

def generate_code(row):
    if pd.notnull(row["surname"]) and len(row["surname"]) >= 3:
        return row["surname"][:3].upper()
    elif pd.notnull(row["forename"]) and len(row["forename"]) >= 3:
        return row["forename"][:3].upper()
    else:
        return (row["forename"] + row["surname"])[:3].upper()

df_drivers["code"] = df_drivers.apply(
    lambda row: generate_code(row) if pd.isnull(row["code"]) else row["code"],
    axis=1
)

print("\nFilled codes (after):")
print(df_drivers[["driverId", "forename", "surname", "code"]])


Missing codes (before):
Empty DataFrame
Columns: [driverId, forename, surname]
Index: []

Filled codes (after):
     driverId  forename     surname code
0           1     Lewis    Hamilton  HAM
1           2      Nick    Heidfeld  HEI
2           3      Nico     Rosberg  ROS
3           4  Fernando      Alonso  ALO
4           5    Heikki  Kovalainen  KOV
..        ...       ...         ...  ...
856       858     Logan    Sargeant  SAR
857       859      Liam      Lawson  LAW
858       860    Oliver     Bearman  BEA
859       861    Franco   Colapinto  COL
860       862      Jack      Doohan  DOO

[861 rows x 4 columns]


In [0]:
# Q4: Youngest and oldest driver per race
# Convert to datetime
df_drivers['dob'] = pd.to_datetime(df_drivers['dob'])
df_races['date'] = pd.to_datetime(df_races['date'])
# Merge drivers with results and race date
df_age = df_results.merge(df_drivers, on='driverId').merge(df_races[['raceId', 'date']], on='raceId')
df_age['age'] = (df_age['date'] - df_age['dob']).dt.days

youngest = df_age.sort_values(['raceId', 'age']).groupby('raceId').first().reset_index()
oldest = df_age.sort_values(['raceId', 'age'], ascending=False).groupby('raceId').first().reset_index()
print("\nQ4: Youngest drivers per race")
print(youngest[['raceId', 'driverId', 'age']].head())
print("\nQ4: Oldest drivers per race")
print(oldest[['raceId', 'driverId', 'age']].head())


Q4: Youngest drivers per race
   raceId  driverId   age
0       1        67  7454
1       2        67  7461
2       3        67  7475
3       4        67  7482
4       5        67  7496

Q4: Oldest drivers per race
   raceId  driverId    age
0       1        22  13459
1       2        22  13466
2       3        22  13480
3       4        22  13487
4       5        22  13501


In [0]:
# Q5: Most wins (positionOrder == 1) and most last places
wins = df_results[df_results['positionOrder'].astype(int) == 1].groupby('driverId').size().sort_values(ascending=False)
lasts = df_results.groupby('raceId')['positionOrder'].transform('max') == df_results['positionOrder']
losses = df_results[lasts].groupby('driverId').size().sort_values(ascending=False)
print("Q5: Most wins")
print(wins.head())
print("\nQ5: Most last places")
print(losses.head())

Q5: Most wins
driverId
1      105
30      91
830     63
20      53
117     51
dtype: int64

Q5: Most last places
driverId
807    19
8      16
154    15
4      14
15     14
dtype: int64


In [0]:
# Q6: Extra question: Average number of laps per driver
lap_counts = df_lap_times.groupby(['raceId', 'driverId'])['lap'].nunique().reset_index(name='num_laps')
avg_laps = lap_counts.groupby('driverId')['num_laps'].mean().reset_index(name='avg_laps')
print("Q6: Average laps per driver")
print(avg_laps)

Q6: Average laps per driver
     driverId   avg_laps
0           1  57.991429
1           2  54.807910
2           3  55.805000
3           4  55.263291
4           5  54.779817
..        ...        ...
138       858  53.083333
139       859  59.090909
140       860  56.666667
141       861  49.625000
142       862  57.000000

[143 rows x 2 columns]
